<a href="https://colab.research.google.com/github/fillobissi/geoinformatics_project/blob/main/250520_Heat_stress_indices_81_00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dask netCDF4 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.3 MB/s eta 0:00:00


In [ ]:
!pip install dask[complete] --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import time

**01 Estrazione delle soglie per ciascun time stamp sui dataset allargati**

In [ ]:
# ------------------ CONFIGURAZIONE ------------------
soglie = {
    "Heat Index": 40.6,
    "Humidex": 45,
    "Lethal Heat Stress Index": 27,
    "UTCI": 46,
    "WBGT": 30,
    "Relative Humidity": 80  # nuova soglia aggiunta
}

# ------------------ FUNZIONI DI CALCOLO ------------------
def calculate_relative_humidity(Ta_c, Td_c):
    return 100.0 * np.exp((17.625 * Td_c) / (243.04 + Td_c) - (17.625 * Ta_c) / (243.04 + Ta_c))

def calculate_humidex(Ta_k, Td_k):
    e = 6.11 * np.exp(5417.7530 * (1 / 273.16 - 1 / Td_k))
    return Ta_k - 273.15 + 0.5555 * (e - 10.0)

def calculate_heat_index(Ta_c, RH):
    T = Ta_c
    HI = -8.784695 + 1.61139411 * T + 2.338549 * RH - 0.14611605 * T * RH \
         - 0.012308094 * T**2 - 0.016424828 * RH**2 + 0.002211732 * T**2 * RH \
         + 0.00072546 * T * RH**2 - 0.000003582 * T**2 * RH**2
    return HI

def calculate_wbt(Ta_c, RH):
    return (Ta_c * np.arctan(0.151977 * np.sqrt(RH + 8.313659))
            + np.arctan(Ta_c + RH) - np.arctan(RH - 1.676331)
            + 0.00391838 * RH**1.5 * np.arctan(0.023101 * RH) - 4.686035)

def calculate_wbgt(Ta_c, Tw):
    return 0.7 * Tw + 0.3 * Ta_c

def calculate_lethal_heat_stress_index(Tw, RH):
    return Tw + 0.1 * RH

def calculate_utci(Ta_c, RH):
    return Ta_c + 0.6 * RH / 100.0

def calculate_stats(data_array):
    return {
        "mean": float(np.nanmean(data_array)),
        "median": float(np.nanmedian(data_array)),
        "p95": float(np.nanpercentile(data_array, 95)),
        "p99": float(np.nanpercentile(data_array, 99)),
        "max": float(np.nanmax(data_array))
    }

# ------------------ CARICAMENTO DEI DATASET ------------------
dataset_temp = xr.open_dataset("/content/drive/MyDrive/Colab Notebooks/hs_datasets/air_temperature_era5-downscaled-over-italy_hourly_81_00.nc")  # T_2M
dataset_dew  = xr.open_dataset("/content/drive/MyDrive/Colab Notebooks/hs_datasets/dew_point_temperature_era5-downscaled-over-italy_hourly_81_00.nc")  # TD_2M

timestamps = dataset_temp['T_2M'].time.values
heatstress_records = []

# ------------------ ELABORAZIONE ------------------
current_year = None
start_time = None

for i, time_selected in enumerate(timestamps):
    ts_year = pd.to_datetime(str(time_selected)).year

    if current_year is None or ts_year != current_year:
        if start_time is not None:
            elapsed = time.time() - start_time
            print(f"  Anno {current_year} completato in {elapsed:.1f} secondi.")
        current_year = ts_year
        start_time = time.time()
        print(f"\n Analizzando l'anno: {current_year}")

    # Snapshot per il timestamp
    Ta_k = dataset_temp['T_2M'].sel(time=time_selected)
    Td_k = dataset_dew['TD_2M'].sel(time=time_selected)

    # Interpolazione dew point se < 243.15K
    Td_k = Td_k.where(Td_k > 243.15)
    Td_k = Td_k.interpolate_na(dim='rlat', method='linear') \
                 .interpolate_na(dim='rlon', method='linear')

    # Conversione °C
    Ta_c = Ta_k - 273.15
    Td_c = Td_k - 273.15

    # Calcolo indici
    RH     = calculate_relative_humidity(Ta_c, Td_c)
    HI     = calculate_heat_index(Ta_c, RH)
    Humidex = calculate_humidex(Ta_k, Td_k)
    WBT     = calculate_wbt(Ta_c, RH)
    WBGT    = calculate_wbgt(Ta_c, WBT)
    LHS     = calculate_lethal_heat_stress_index(WBT, RH)
    UTCI    = calculate_utci(Ta_c, RH)

    # Raccolta indici
    indices = {
        "Heat Index": HI,
        "Humidex": Humidex,
        "Lethal Heat Stress Index": LHS,
        "UTCI": UTCI,
        "WBGT": WBGT,
        "Relative Humidity": RH
    }

    # Calcolo statistiche
    stats_results = {key: calculate_stats(val) for key, val in indices.items()}

    # Costruzione DataFrame
    df_results = pd.DataFrame([
        {
            "Indice": key,
            "Media (°C)": val["mean"],
            "Mediana (°C)": val["median"],
            "95° Perc. (°C)": val["p95"],
            "99° Perc. (°C)": val["p99"],
            "Massimo (°C)": val["max"],
            "Soglia (°C)": soglie[key],
            "Media > Soglia": "si" if val["mean"] > soglie[key] else "no",
            "Mediana > Soglia": "si" if val["median"] > soglie[key] else "no",
            "95° Perc. > Soglia": "si" if val["p95"] > soglie[key] else "no",
            "99° Perc. > Soglia": "si" if val["p99"] > soglie[key] else "no",
            "Massimo > Soglia": "si" if val["max"] > soglie[key] else "no"
        }
        for key, val in stats_results.items()
    ])

    df_results.insert(0, "Timestamp", str(time_selected))
    df_results.insert(1, "Numero Timestamp", i + 1)
    heatstress_records.extend(df_results.to_dict(orient="records"))

# ------------------ OUTPUT ------------------
df_heatstress = pd.DataFrame(heatstress_records)
df_heatstress["Data"] = pd.to_datetime(df_heatstress["Timestamp"]).dt.date

# Esporta anche in Excel
output_excel = "heatstress_all_timestamps_all_years.xlsx"
df_heatstress.to_excel(output_excel, index=False)

print(f"File Excel creato con successo: {output_excel}")


🚀 Analizzando l'anno: 1981
 ✅ Anno 1981 completato in 194.4 secondi.

🚀 Analizzando l'anno: 1982
 ✅ Anno 1982 completato in 197.3 secondi.

🚀 Analizzando l'anno: 1983
 ✅ Anno 1983 completato in 191.2 secondi.

🚀 Analizzando l'anno: 1984
 ✅ Anno 1984 completato in 194.3 secondi.

🚀 Analizzando l'anno: 1985
 ✅ Anno 1985 completato in 186.7 secondi.

🚀 Analizzando l'anno: 1986
 ✅ Anno 1986 completato in 190.6 secondi.

🚀 Analizzando l'anno: 1987
 ✅ Anno 1987 completato in 191.6 secondi.

🚀 Analizzando l'anno: 1988
 ✅ Anno 1988 completato in 189.6 secondi.

🚀 Analizzando l'anno: 1989
 ✅ Anno 1989 completato in 190.3 secondi.

🚀 Analizzando l'anno: 1990
 ✅ Anno 1990 completato in 188.1 secondi.

🚀 Analizzando l'anno: 1991
 ✅ Anno 1991 completato in 192.0 secondi.

🚀 Analizzando l'anno: 1992
 ✅ Anno 1992 completato in 189.4 secondi.

🚀 Analizzando l'anno: 1993
 ✅ Anno 1993 completato in 193.1 secondi.

🚀 Analizzando l'anno: 1994
 ✅ Anno 1994 completato in 191.5 secondi.

🚀 Analizzando l'ann

In [ ]:
import pandas as pd
import os

# 1. Legge il file Excel generato
excel_path = "/content/heatstress_all_timestamps_all_years.xlsx"
df = pd.read_excel(excel_path)

# 2. Crea la cartella di destinazione (se non esiste)
output_dir = "/content/miei_csv"
os.makedirs(output_dir, exist_ok=True)

# 3. Scrive il CSV nella directory scelta
csv_path = os.path.join(output_dir, "heatstress_all_timestamps_all_years.csv")
df.to_csv(csv_path, index=False)

print(f"✅ File CSV salvato in: {csv_path}")


✅ File CSV salvato in: /content/miei_csv/heatstress_all_timestamps_all_years.csv


In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import os
import time

# ------------------ CONFIGURAZIONE ------------------
soglie = {
    "Heat Index": 40.6,
    "Humidex": 45,
    "Lethal Heat Stress Index": 27,
    "UTCI": 46,
    "WBGT": 30,
    "Relative Humidity": 80
}

# ------------------ FUNZIONI DI CALCOLO ------------------
def calculate_relative_humidity(Ta_c, Td_c):
    return 100.0 * np.exp((17.625 * Td_c) / (243.04 + Td_c) - (17.625 * Ta_c) / (243.04 + Ta_c))

def calculate_humidex(Ta_k, Td_k):
    e = 6.11 * np.exp(5417.7530 * (1 / 273.16 - 1 / Td_k))
    return Ta_k - 273.15 + 0.5555 * (e - 10.0)

def calculate_heat_index(Ta_c, RH):
    T = Ta_c
    return (-8.784695 + 1.61139411 * T + 2.338549 * RH - 0.14611605 * T * RH
            - 0.012308094 * T**2 - 0.016424828 * RH**2 + 0.002211732 * T**2 * RH
            + 0.00072546 * T * RH**2 - 0.000003582 * T**2 * RH**2)

def calculate_wbt(Ta_c, RH):
    return (Ta_c * np.arctan(0.151977 * np.sqrt(RH + 8.313659))
            + np.arctan(Ta_c + RH) - np.arctan(RH - 1.676331)
            + 0.00391838 * RH**1.5 * np.arctan(0.023101 * RH) - 4.686035)

def calculate_wbgt(Ta_c, Tw):
    return 0.7 * Tw + 0.3 * Ta_c

def calculate_lethal_heat_stress_index(Tw, RH):
    return Tw + 0.1 * RH

def calculate_utci(Ta_c, RH):
    return Ta_c + 0.6 * RH / 100.0

def calculate_stats(data_array):
    return {
        "mean": float(np.nanmean(data_array)),
        "median": float(np.nanmedian(data_array)),
        "p95": float(np.nanpercentile(data_array, 95)),
        "p99": float(np.nanpercentile(data_array, 99)),
        "max": float(np.nanmax(data_array))
    }

# ------------------ CARICAMENTO FILE CSV ESISTENTE ------------------
csv_path = "/content/drive/MyDrive/Colab Notebooks/hs_outputs/heatstress_all_timestamps_all_years.csv"

if not os.path.exists(csv_path):
    raise FileNotFoundError(f"❌ File CSV non trovato in: {csv_path}. Interruzione esecuzione.")

df_existing = pd.read_csv(csv_path)
print(f"✅ CSV esistente caricato: {len(df_existing)} righe.")

# ------------------ CARICAMENTO DEI NUOVI DATASET ------------------
dataset_temp = xr.open_dataset("/content/drive/MyDrive/Colab Notebooks/hs_datasets/air_temperature_era5-downscaled-over-italy_hourly_01_23.nc")
dataset_dew  = xr.open_dataset("/content/drive/MyDrive/Colab Notebooks/hs_datasets/dew_point_temperature_era5-downscaled-over-italy_hourly_01_23.nc")

timestamps = dataset_temp['T_2M'].time.values
heatstress_records = []

# ------------------ ELABORAZIONE ------------------
current_year = None
start_time = None

for i, time_selected in enumerate(timestamps):
    ts_year = pd.to_datetime(str(time_selected)).year

    if current_year is None or ts_year != current_year:
        if start_time is not None:
            elapsed = time.time() - start_time
            print(f"  ✅ Anno {current_year} completato in {elapsed:.1f} secondi.")
        current_year = ts_year
        start_time = time.time()
        print(f"\n🚀 Analizzando l'anno: {current_year}")

    # Snapshot
    Ta_k = dataset_temp['T_2M'].sel(time=time_selected)
    Td_k = dataset_dew['TD_2M'].sel(time=time_selected)
    Td_k = Td_k.where(Td_k > 243.15).interpolate_na(dim='rlat').interpolate_na(dim='rlon')

    Ta_c = Ta_k - 273.15
    Td_c = Td_k - 273.15

    RH     = calculate_relative_humidity(Ta_c, Td_c)
    HI     = calculate_heat_index(Ta_c, RH)
    Humidex = calculate_humidex(Ta_k, Td_k)
    WBT     = calculate_wbt(Ta_c, RH)
    WBGT    = calculate_wbgt(Ta_c, WBT)
    LHS     = calculate_lethal_heat_stress_index(WBT, RH)
    UTCI    = calculate_utci(Ta_c, RH)

    indices = {
        "Heat Index": HI,
        "Humidex": Humidex,
        "Lethal Heat Stress Index": LHS,
        "UTCI": UTCI,
        "WBGT": WBGT,
        "Relative Humidity": RH
    }

    stats_results = {key: calculate_stats(val) for key, val in indices.items()}

    df_results = pd.DataFrame([
        {
            "Indice": key,
            "Media (°C)": val["mean"],
            "Mediana (°C)": val["median"],
            "95° Perc. (°C)": val["p95"],
            "99° Perc. (°C)": val["p99"],
            "Massimo (°C)": val["max"],
            "Soglia (°C)": soglie[key],
            "Media > Soglia": "si" if val["mean"] > soglie[key] else "no",
            "Mediana > Soglia": "si" if val["median"] > soglie[key] else "no",
            "95° Perc. > Soglia": "si" if val["p95"] > soglie[key] else "no",
            "99° Perc. > Soglia": "si" if val["p99"] > soglie[key] else "no",
            "Massimo > Soglia": "si" if val["max"] > soglie[key] else "no"
        }
        for key, val in stats_results.items()
    ])

    df_results.insert(0, "Timestamp", str(time_selected))
    df_results.insert(1, "Numero Timestamp", i + 1)
    heatstress_records.extend(df_results.to_dict(orient="records"))

# ------------------ UNIONE ED ESPORTAZIONE ------------------
df_new = pd.DataFrame(heatstress_records)
df_new["Data"] = pd.to_datetime(df_new["Timestamp"]).dt.date

# Rimozione duplicati (se timestamp già esistono)
timestamps_existing = set(df_existing["Timestamp"].astype(str))
df_new = df_new[~df_new["Timestamp"].isin(timestamps_existing)]

df_combined = pd.concat([df_existing, df_new], ignore_index=True)
df_combined.to_csv(csv_path, index=False)

print(f"\n✅ File CSV aggiornato con successo: {csv_path}")
print(f"📦 Totale righe finali: {len(df_combined)}")


✅ CSV esistente caricato: 306810 righe.

🚀 Analizzando l'anno: 2001
  ✅ Anno 2001 completato in 201.4 secondi.

🚀 Analizzando l'anno: 2002
  ✅ Anno 2002 completato in 198.2 secondi.

🚀 Analizzando l'anno: 2003
  ✅ Anno 2003 completato in 192.8 secondi.

🚀 Analizzando l'anno: 2004
  ✅ Anno 2004 completato in 200.1 secondi.

🚀 Analizzando l'anno: 2005
  ✅ Anno 2005 completato in 191.2 secondi.

🚀 Analizzando l'anno: 2006
  ✅ Anno 2006 completato in 194.9 secondi.

🚀 Analizzando l'anno: 2007
  ✅ Anno 2007 completato in 194.3 secondi.

🚀 Analizzando l'anno: 2008
  ✅ Anno 2008 completato in 198.5 secondi.

🚀 Analizzando l'anno: 2009
  ✅ Anno 2009 completato in 192.6 secondi.

🚀 Analizzando l'anno: 2010
  ✅ Anno 2010 completato in 195.8 secondi.

🚀 Analizzando l'anno: 2011
  ✅ Anno 2011 completato in 194.4 secondi.

🚀 Analizzando l'anno: 2012
  ✅ Anno 2012 completato in 197.0 secondi.

🚀 Analizzando l'anno: 2013
  ✅ Anno 2013 completato in 191.0 secondi.

🚀 Analizzando l'anno: 2014
  ✅ Anno 